In [2]:
import numpy as np
from datetime import datetime
import pandas as pd
import requests
from tqdm import tqdm
pd.set_option('max_colwidth', None) # show all text in a column
pd.set_option('display.max_columns', None) # show all columns

tqdm.pandas()

import aiohttp
import asyncio

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

/Users/isaacduong/BMBF_Clustering_Project/BMBFenv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
BMBF = pd.read_csv("../data/cleanedBMBF.csv")

In [6]:
BMBF

,FKZ,Ressort,Referat,PT,Arb.-Einh.,Zuwendungsempfänger,Gemeindekennziffer,Stadt/Gemeinde,Ort,Bundesland,Staat,Ausführende Stelle,Gemeindekennziffer.1,Stadt/Gemeinde.1,Ort.1,Bundesland.1,Staat.1,Thema,Leistungsplansystematik,Klartext Leistungsplansystematik,Laufzeit von,Laufzeit bis,Fördersumme in EUR,Förderprofil,Verbundprojekt,Förderart
0,03F0212C/7,BMBF,724,PT-J,MGS1,Universität Rostock,13003000.0,Rostock,Rostock,Mecklenburg-Vorpommern,Deutschland,Universität Rostock - Fachbereich Elektrotechnik und Informationstechnik,13003000.0,Rostock,Rostock,Mecklenburg-Vorpommern,Deutschland,"Verbundprojekt MESSIN: Entwicklung und Systemintegration der Komponenten Navigation, Automatische Steuerung, Kommunikation und Energieversorgung für den Meßdelphin MESSIN",FB4010,"Technik-/Infrastrukturentwicklung und -bereitstellung (Überwachungssysteme, Begleitforschung zum Monitoring, innovative Technik und Geräteträger)",01.01.1998,31.08.2000,"648.462,00",Forschung und Entwicklung zur Daseinsvorsorge,MESSIN,PDIR
1,D251800/0,BMBF,314,BIBB,A3.3,"Ministerium für Wirtschaft, Arbeit und Wohnungsbau Baden-Württemberg",8111000.0,Stuttgart,Stuttgart,Baden-Württemberg,Deutschland,KACO new energy GmbH,8121000.0,Heilbronn,Heilbronn,Baden-Württemberg,Deutschland,Prozeßorientierte Entwicklungsplanung und Qualifizierung im Betrieb,OB1050,Versuchs- und Modelleinrichtungen und -programme im Bereich der beruflichen Bildung,01.07.1997,30.06.2000,"96.634,00",Forschung und Entwicklung zur Daseinsvorsorge,NaN,PDIR
2,D251800B/2,BMBF,314,BIBB,A3.3,"Ministerium für Wirtschaft, Arbeit und Wohnungsbau Baden-Württemberg",8111000.0,Stuttgart,Stuttgart,Baden-Württemberg,Deutschland,CALA e.V.,8118019.0,Gerlingen,Gerlingen,Baden-Württemberg,Deutschland,Wiss. Begl. zum MV 'Prozeßorientierte Entwicklungsplanung und Qualifizierung im Betrieb,OB1050,Versuchs- und Modelleinrichtungen und -programme im Bereich der beruflichen Bildung,01.07.1997,30.06.2000,"157.457,00",Forschung und Entwicklung zur Daseinsvorsorge,NaN,PDIR
3,50WM9821/6,BMWi,IVD5,DLR-RFM,AR-AF,Fraunhofer-Gesellschaft zur Förderung der angewandten Forschung eingetragener Verein,9162000.0,München,München,Bayern,Deutschland,Fraunhofer-Gesellschaft zur Förderung der angewandten Forschung eingetragener Verein,9162000.0,München,München,Bayern,Deutschland,Untersuchung des Schäumens von Metallen unter variierten Schwerkraftbedingungen,IB3010,Forschung unter Weltraumbedingungen - Materialwissenschaften,01.03.1998,31.12.2001,"452.356,00",Technologie- und Innovationsförderung,NaN,PDIR
4,50WB9716/8,BMWi,IVD5,DLR-RFM,AR-AF,Universität Hohenheim,8111000.0,Stuttgart,Stuttgart,Baden-Württemberg,Deutschland,Universität Hohenheim - Fakultät Naturwissenschaften - Institut für Physiologie,8111000.0,Stuttgart,Stuttgart,Baden-Württemberg,Deutschland,Der Einfluß verschiedener Gravitationsbedingungen auf das Verhalten künstlicher und nativer Ionenkanäle,IB3020,Forschung unter Weltraumbedingungen - Biowissenschaften und Medizin,01.01.1998,31.03.2002,"358.551,00",Technologie- und Innovationsförderung,NaN,PDIR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272239,03EN4022J,BMWi,IIC6,PT-J,ESN4,Georgsmarienhütte GmbH,3459019.0,Georgsmarienhütte,Georgsmarienhütte,Niedersachsen,Deutschland,Georgsmarienhütte GmbH,3459019.0,Georgsmarienhütte,Georgsmarienhütte,Niedersachsen,Deutschland,"Verbundvorhaben: SiPro - Simulationsbasiertes Eigenschaftsdesign entlang der kombinierten Prozesskette Ur- und Umformtechnik, Teilvorhaben: Herstellung der industriellen Schmelzen über die Strangguss/Blockguss-Route bis zum Warmwalzen",EA3251,Energiesparende Industrieverfahren - Fertigungstechnik,01.01.2022,31.12.2024,"227.401,00",Technologie- und Innovationsförderung,SiPro,PDIR
272240,67K18622,BMU,IKIII3,PT-ZUG,NKI,Amt Bokhorst-Wankendorf,1057085.0,Wankendorf,Wankendorf,Schleswig-Holstein,Deutschland,Amt Bokhorst-Wankendorf,1057085.0,Wankendorf,Wankendorf,Schleswig-Holstein,Deutschland,KSI: Sanierung der I

### For retrieving publications and press releases on idw , FKZ alone can not be used for finding press releases and publications related to the BMBF grants. Therefore all press releases and publications mentioned BMBF are retrieved at first. After that FKZ will be using to find matches.

In [ ]:
def extract_pub_web_address(soup):
    # this function extract html links
    div_elements = soup.find_all("div", class_="nine columns")
    web_address = []
    if div_elements:
        for div in div_elements:
            # Find and extract the <a> element within the div
            a_element = div.find("a")

            if a_element:
                # Extract the 'href' attribute value from the <a> element
                href_value = a_element.get("href")
                web_address.append(href_value)
                
    return web_address

In [ ]:
search_term = "BMBF"  # search term BMBF returns 21979 articles as well as 1099 pages whereas search term "Bundesministerium für Bildung und Forschung" returns only 1 result

def retrieve_pub_web_address(search_term, pages = 1099):
    # this function go over all pages and extract all links 
    
    url = f"https://idw-online.de/de/simplesearch?words={search_term}&_form_=InputForm&scope=press_release&scope=event&page=%s"
    pub_web_addresses = []
    # Send an HTTP GET request to the URL
    for page in tqdm(range(1, pages + 1)):
        response = requests.get(url % page)
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content of the page
            soup = BeautifulSoup(response.text, "html.parser")

            pub_web_addresses.extend(extract_pub_web_address(soup))

    return pub_web_addresses

In [ ]:
web_addresses = retrieve_pub_web_address(search_term)

In [ ]:
%store web_addresses

In [ ]:
def scrape_text_by_xpath(url, xpath):
    # Specify the path to the ChromeDriver executable
    chrome_driver_path = "../chromedriver"

    # Create a Service object with the ChromeDriver executable path
    service = Service(chrome_driver_path)
    chrome_options = webdriver.ChromeOptions()
    chrome_options.headless = True

    driver = webdriver.Chrome(options=chrome_options, service=service)

    try:
        # Open the URL in the browser
        driver.get(url)

        # Find the element using the provided XPath
        element = driver.find_element(By.XPATH, xpath)

        # Extract the text from the element
        text = element.text

        return text

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the browser window
        driver.quit()

### webscraping by selenium take long time to scrape a press release ( about 20 seconds each item ), therefore we asynchronously retrieving contents using moduls requests and beautifulsoup) in follwing cell
def get_press_content(web_address):
    
    title = "/html/body/div[5]/div[2]/div/h4[2]"
    sub_title ="/html/body/div[5]/div[2]/div/h5"
    content = "/html/body/div[5]/div[2]/div/p[3]"

    scraped_text = ""
    url="https://idw-online.de" + web_address
    
    for xpath in [title,sub_title,content]:
        scraped_text += scrape_text_by_xpath(url, xpath)

    if scraped_text:
        return scraped_text
    else:
        print("No text found at the specified XPath.")
        return None


% time get_press_content(web_addresses[0])

In [ ]:
async def fetch_url(session, press_web_address, timeout=20):
    url = "https://idw-online.de" + press_web_address

    try:
        async with session.get(url) as response:
            if response.status == 200:
                html_content = await response.text()
                return press_web_address, html_content

    except Exception as e:
        print(e)
        return None, None


async def fetch_data_from_urls(urls):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in tqdm(urls):
            tasks.append(fetch_url(session, url))
        results = await asyncio.gather(*tasks)
        return results

In [ ]:
chunk_size = 100
results = []
for chunk in tqdm(range(0, len(web_addresses), chunk_size)):
    start = chunk
    end = min(chunk + chunk_size, len(web_addresses))
    start_time = datetime.now()
    result = await fetch_data_from_urls(web_addresses[start:end])
    results.append(result)
    time = datetime.now() - start_time
    print(time)

In [ ]:
#create a DataFrame from retrieving data
idw = pd.DataFrame(results[0])
for i in range(1, len(results)):
    idw = pd.concat([idw, pd.DataFrame(results[i])], ignore_index=True)
idw.columns = ["web_address", "contents"]

In [ ]:
idw.dropna(inplace=True)

In [ ]:
#select only relevant paragraph 
def select_content(row):
    return row["contents"].split('<h5 class="subheader">')[1].split("style")[0]


idw = idw.progress_apply(select_content, axis=1)

In [ ]:
idw.to_csv("../data/idw_enrichment.csv")

In [ ]:
# remove html tags
idw = idw.apply(lambda html_text: re.sub(r"<[^>]*>", "", html_text))

In [ ]:
FKZs_with_pubs = []

### Running follwing cell will takes about 3 hours

In [29]:
# this cell will determine those FKZs which are mentioned in press release/publication retrieved
total_contents = "".join(idw.iloc[:,1].values)
print(len(total_contents))
for FKZ in tqdm(BMBF.loc[BMBF["Ressort"] == "BMBF", "FKZ"].values):
  
    if FKZ in total_contents:
        print(f"found publication of FKZ {FKZ}")
        FKZs_with_pubs.append(FKZ)

100305537


  4%|███                                                                        | 7247/174414 [09:43<4:17:24, 10.82it/s]

found publication of FKZ 1104506


 31%|██████████████████████▌                                                 | 54520/174414 [1:00:02<1:43:24, 19.32it/s]

found publication of FKZ 05KS7PC2


...

100%|████████████████████████████████████████████████████████████████████████▊| 174108/174414 [2:42:15<00:14, 21.78it/s]

found publication of FKZ 13N15995


100%|████████████████████████████████████████████████████████████████████████▉| 174123/174414 [2:42:16<00:13, 21.24it/s]

found publication of FKZ 01IS21046B


100%|█████████████████████████████████████████████████████████████████████████| 174414/174414 [2:42:30<00:00, 17.89it/s]


In [30]:
print(f"{len(FKZs_with_pubs)} FKZs have related press release/ publication")

349 FKZs have related press release/ publication


A closer examination of the FKZs reveals that there is often a space between many of them. So lets take an attempt to correct them to see whether we can find press release/publication for those FKZs. 

In [31]:
for FKZ in tqdm(BMBF.loc[BMBF["Ressort"] == "BMBF", "FKZ"].values):
    
    if (FKZ != FKZ.replace(" ","")) and (FKZ.replace(" ","") in total_contents):
        print(f"found publication of FKZ {FKZ}")
        FKZs_with_pubs.append(FKZ)

 36%|███████████████████████████▎                                                | 62702/174414 [23:15<50:39, 36.75it/s]

found publication of FKZ 1304899 


100%|██████████████████████████████████████████████████████████████████████████| 174414/174414 [24:10<00:00, 120.26it/s]


In [89]:
# save the list for late use sothat we dont have to run the notebook from beginning
%store FKZs_with_pubs

Stored 'FKZs_with_pubs' (list)


found 1 more FKZ, which has press release/publication related to it after remove space in FKZ. Total 350 FKZs

In [87]:
#this cell will map the press releases/ publications to related FKZs  
BMBF.loc[:,"idw_enrichment"] = ""
for FKZ in tqdm(FKZs_with_pubs):
    for row in range(len(idw)):
        if FKZ in idw.iloc[row,1]:
            BMBF.loc[BMBF["FKZ"]==FKZ,"idw_enrichment"] += idw.iloc[row,1].replace("  ","").replace("\n","")
                       

100%|█████████████████████████████████████████████████████████████████████████████████| 350/350 [03:06<00:00,  1.88it/s]


In [91]:
BMBF[BMBF["FKZ"].isin(FKZs_with_pubs)]["idw_enrichment"]

10078     Johannes Faber Bernstein KoordinationsstelleNa...
65774     Dr. Josef König PressestelleRuhr-Universität B...
75558                                                      
76365     Doris Böhme Presse- und ÖffentlichkeitsarbeitH...
77468     Dr. Stefan Kurze KommunikationInstitut für Mik...
                                ...                        
269979    NinaReckendorf Stabsstelle Presse, Kommunikati...
270335    Friederike Süssig-Jeschor PressestelleUniversi...
270539    ChristianColmer Press &amp; Public RelationsFr...
271304    Melanie Löw UniversitätskommunikationTechnisch...
271331    Susanne Krause Externe und interne Kommunikati...
Name: idw_enrichment, Length: 350, dtype: object

In [100]:
#set all other field to NaN value 
BMBF.loc[~BMBF["FKZ"].isin(FKZs_with_pubs),"idw_enrichment"]=np.nan

In [101]:
BMBF["idw_enrichment"].isnull().sum()

271894

In [103]:
BMBF.to_csv("../data/BMBF_idw_enrichment.csv", index= False)